In [ ]:
#importing necessary libraries

import pytest
from io import StringIO
from contextlib import redirect_stdout
from mining_module import OreDeposit, DrillingEquipment, HaulingEquipment, MiningSimulation

# Importing mining simulation classes and functions 

@pytest.fixture
def data():
    return {
        'ore_name': 'gold',
        'total_reserve': '11556',
        'average_grade': '4.56',
        'Equipment': 'Excavator',
        'mining_rate': '0.63',
        'drilling_equipment_name': 'Drill1',
        'drilling_rate': '0.6',
        'hauling_equipment_name': 'Truck1',
        'hauling_rate': '0.75',
        'haul_capacity': '100',
        'Fuel_consumed': '210',
        'Depth': '200',
        'Length': '1000',
        'Width': '800'
    }

def test_surface_mine_simulation(data):
    # Capture the printed output for analysis
    captured_output = StringIO()
    with redirect_stdout(captured_output):
        # Simulate surface mine scenario
        surface_mine_simulation = MiningSimulation(
            mine_section=SurfaceMineSection(
                depth=float(data["Depth"]),
                length=float(data["Length"]),
                width=float(data["Width"])
            )
        )

        surface_mine_simulation.simulate_extraction(time_period=1)
        surface_mine_simulation.simulate_hauling(time_period=1)

    # Reset redirect.
    captured_output.seek(0)
    output = captured_output.getvalue()

    # Perform assertions based on the expected output or result
    assert "Surface Mine Simulation Completed:" in output
    assert "tons of ore extracted from Surface Mine." in output
    assert "tons of ore hauled from Surface Mine." in output

def test_underground_mine_simulation(data):
    # Capture the printed output for analysis
    captured_output = StringIO()
    with redirect_stdout(captured_output):
        # Simulate underground mine scenario
        underground_mine_simulation = MiningSimulation(
            mine_section=UndergroundMineSection(
                depth=float(data["Depth"]),
                length=float(data["Length"]),
                width=float(data["Width"])
            )
        )

        underground_mine_simulation.simulate_extraction(time_period=1)

    # Reset redirect.
    captured_output.seek(0)
    output = captured_output.getvalue()

    # Performing assertions based on the expected output 
    assert "Underground Mine Simulation Completed:" in output
    assert "tons of ore extracted from Underground Mine." in output

In [ ]:
#importing required libraries
import pytest
import pandas as pd
from mining_module import MiningEquipment, MiningSimulation  

@pytest.fixture
def data():
    
    return pd.DataFrame({
        'Equipment': ['Drill1'],
        'Capacity': [100],
        'Power': [50]
    })

def test_mining_simulation(sample_data):#function for testing mining simulation
    equipment = MiningEquipment(
        name=data['Equipment'][0],
        equipment_capacity=data['Capacity'][0],
        equipment_power=data['Power'][0]
    )

    mine_simulation = MiningSimulation(mining_equipment=equipment)
    equipment_specs = mine_simulation.mining_equipment.get_equipment_specs()

    assert equipment.name == 'Excavator'
    assert equipment_specs['capacity'] == 100
    assert equipment_specs['power'] == 50







In [ ]:
#importing required libraries
import pytest
from mining_module import OreDeposit, MiningEquipment, DrillingEquipment, HaulingEquipment, MiningSimulation, read_data_from_file

@pytest.fixture
def data():
    return {
        'ore_name': 'Gold',
        'total_reserve': '1000.0',
        'average_grade': '5.0',
        'Equipment': 'Excavator',
        'mining_rate': '10.0',
        'drilling_equipment_name': 'Drill1',
        'drilling_rate': '5.0',
        'hauling_equipment_name': 'Truck1',
        'hauling_rate': '20.0',
        'haul_capacity': '100.0',
        'Fuel_consumed': '2.0'
    }
@pytest.fixture
def mining_simulation(data):#function for testing mining simulation
    gold_deposit = OreDeposit(
        name=data['ore_name'],
        total_reserve=float(data['total_reserve']),
        average_grade=float(data['average_grade'])
    )
    excavator = MiningEquipment(
        name=data['Equipment'],
        mining_rate=float(data['mining_rate'])
    )
    drill = DrillingEquipment(
        name=data['drilling_equipment_name'],
        drilling_rate=float(data['drilling_rate'])
    )
    truck = HaulingEquipment(
        name=data['hauling_equipment_name'],
        hauling_rate=float(data['hauling_rate']),
        capacity=float(data['haul_capacity']),
        fuel_consumption_rate=float(data['Fuel_consumed'])
    )
    mine_simulation = MiningSimulation(
        ore_deposit=gold_deposit,
        mining_equipment=excavator,
        drilling_equipment=drill,
        hauling_equipment=truck
    )
    return mine_simulation
def test_simulation_extraction_and_hauling(mining_simulation):#function for testing simulation extraction for hauling
    for _ in range(10):
        mining_simulation.simulate_extraction(time_period=1)
        mining_simulation.simulate_hauling(time_period=1)
    extraction_status = mining_simulation.get_extraction_status()
    equipment_statuses = mining_simulation.get_equipment_statuses()
    assert extraction_status['current_reserve'] >= 0
    assert extraction_status['extracted_ore'] >= 0
    assert 0 <= equipment_statuses['hauling_status']['current_fuel_level'] <= 100

In [ ]:
import pytest
from mining_module import Environment, MiningSimulation, read_data_from_file
# Define a fixture to read data from a test CSV file
@pytest.fixture
def test_environments():
    return read_data_from_file('geo.csv')
# Test the Environment class
def test_environment_specs(test_environments):
    for environment in test_environments:
        specs = environment.get_environment_specs()
        assert isinstance(specs['depth'], float)
        assert isinstance(specs['length'], float)
        assert isinstance(specs['width'], float)
# Test the MiningSimulation class
def test_mining_simulation():
    environment = Environment(type='Test', depth=10.0, length=20.0, width=30.0)
    simulation = MiningSimulation(environment)
    assert simulation.environment.type == 'Test'
# Test the read_data_from_file function
def test_read_data_from_file():
    environments = read_data_from_file('geo.csv')
    assert len(environments) > 0
    for environment in environments:
        assert isinstance(environment, Environment)



In [ ]:
import pytest
from unittest.mock import patch#patch to mock the behaviour of print
from mining_module import Environment, MiningSimulation, read_data_from_file
# Define a fixture to provide sample environments
@pytest.fixture
def sample_environments():
    return [
        Environment(type='Sample1', depth=10.0, length=20.0, width=30.0),
        Environment(type='Sample2', depth=15.0, length=25.0, width=35.0)
    ]
# Test the drill method of MiningSimulation
def test_drill(sample_environments, capsys):
    for environment in sample_environments:
        simulation = MiningSimulation(environment)
        simulation.drill()
        captured = capsys.readouterr()
        assert f"Drilling operation initiated in {environment.type}." in captured.out
# Test the haul method of MiningSimulation
def test_haul(sample_environments, capsys):
    for environment in sample_environments:
        simulation = MiningSimulation(environment)
        simulation.haul()
        captured = capsys.readouterr()